## Deep Convolutional GAN (DCGAN)

### Introduction
Generative Adversarial Networks, or GANs, are a class of machine-learning models that learn by competition. One network, the *generator*, proposes candidate images; the other, the *discriminator*, evaluates whether each proposal resembles the genuine training data. Through iterative feedback the generator refines its output while the discriminator sharpens its judgement, producing a virtuous cycle that can yield highly realistic synthetic images.

GANs were a conceptual breakthrough but lacked a mechanism for recognising spatial structure, so it produced images with limited detail. A subsequent design, the Deep Convolutional GAN (DCGAN), replaced dense layers with convolutional ones that are sensitive to local patterns such as edges and textures. This architectural change greatly improves visual fidelity while keeping computational demands modest, which makes the DCGAN a common introductory model in academic settings.

Further variants address specific research questions. Conditional GANs incorporate class labels so that the generator can be instructed to create a particular category of image. Pix2Pix learns direct mappings between paired images, for example line drawings to photographs, whereas CycleGAN achieves similar translations without paired examples by enforcing consistency when images are converted back to their original domain. StyleGAN, a more recent contribution, separates high-level structure from fine detail, enabling photorealistic synthesis of faces and other complex subjects.

These advances are the basis for both creative applications and controversial ones. Deep-fake systems, which replace a person’s face in video footage with a synthetically generated counterpart, rely on GAN architectures that preserve fine detail and temporal coherence. Such technology can streamline film production and accessibility work but also raises ethical concerns regarding misinformation and personal privacy, prompting ongoing research into watermarking, authenticity verification and regulatory frameworks.

### Fashion MNIST dataset

We base our experiments on the Fashion-MNIST corpus, an openly available benchmark released by researchers at Zalando Research as a modern alternative to the original handwritten-digit MNIST set. The collection comprises seventy thousand grey-scale raster images, each exactly 28X28 pixels and stored as an eight-bit intensity matrix. The data are already split into 60,000 training examples and 10,000 test examples.

Every image shows a single article of clothing photographed against a plain background. The ten categories: T-shirt/top, trouser, pullover, dress, coat, sandal, shirt, sneaker, bag, and ankle boot, are evenly represented, providing a balanced classification challenge and a diverse visual catalogue for generative work. Although the resolution is deliberately small, local texture such as seams, collars and laces remain visible, giving a model enough structure to learn meaningful patterns.

Fashion-MNIST retains the same dimensions and file format as the digit dataset it replaces, so it serves as a convenient drop-in benchmark. The task is perceptibly harder than identifying digits.


### Install Python libraries

In [ ]:
!pip install tensorflow matplotlib numpy

### Load the dataset
We first call Keras’s helper that downloads Fashion-MNIST and returns two tuples, one for training data and one for test data, each containing images and labels. We discard the entire test set with an underscore, this means our code keeps only the 60,000 training images and deliberately ignores both sets of labels and all test images. This assignment is convenient when the task is unsupervised generation, where only the raw pictures are needed:

In [ ]:
import tensorflow as tf
from tensorflow import keras

(x_train, _), _ = keras.datasets.fashion_mnist.load_data()

### Preprocessing
We next convert the training images from unsigned 8-bit integers to 32-bit floating-point numbers, then rescale each pixel value from its original range 0–255 to the range −1 to +1.

Subtracting 127.5 centres the data at zero, and dividing by 127.5 halves the span so that black becomes −1, white becomes +1, and mid-grey becomes 0.

This kind of normalisation is standard when the generator’s final activation is `tanh`, because the network learns more smoothly when its inputs and outputs share the same numerical range.


In [ ]:
x_train = (x_train.astype('float32') - 127.5) / 127.5

Next, we set a number of constants, including:

- `LATENT_DIM = 100` sets the length of the random "seed" vector that the generator turns into an image. A hundred numbers give the model enough room to encode varied styles and shapes without making training too heavy.

- `BUFFER_SIZE = 60000` matches the number of training pictures. When we shuffle the dataset, TensorFlow draws from a buffer this large, ensuring that every image can appear in any order each epoch.

- `BATCH_SIZE = 64` tells the training loop to process sixty-four images at a time. This mini-batch size offers a practical balance: it is small enough to fit comfortably in GPU memory yet large enough to provide stable gradient estimates for learning.

In [ ]:
 # Global constants
LATENT_DIM  = 100
BUFFER_SIZE = 60000
BATCH_SIZE  = 64

### Resampling
Next we add an extra channel dimension to every picture so that each image is treated as 28 X 28 pixels with one grey-scale layer, rather than a simple 28 X 28 grid. Convolutional image layers expect that extra dimension, even when the pictures are black and white.

The next part of our code turns the array into a streaming data set for TensorFlow. It breaks the images into individual records, mixes them up so the model does not see them in the same order every time, and then bundles them into equal-sized mini-batches. Any leftovers that do not fit neatly into a batch are dropped:

In [ ]:
import numpy as np
# Add a channel dimension so each image is (28, 28, 1)
x_train = np.expand_dims(x_train, -1)

# Build a TensorFlow input pipeline
train_ds = (
    tf.data.Dataset
      # Create a dataset where each element is one image tensor
      .from_tensor_slices(x_train)
      # Shuffle the order of images at every epoch to prevent learning order biases
      .shuffle(BUFFER_SIZE)
      # Group images into equal-sized mini-batches; drop remainder for clean shape
      .batch(BATCH_SIZE, drop_remainder=True)
)


### Generator
The `build_generator` function constructs the network that will *invent* new images. It begins with an input layer that accepts a latent vector of length `LATENT_DIM`, a string of random numbers that acts as the creative seed. A fully connected (`Dense`) layer then projects this one-dimensional seed into 7 X 7 X 128 numerical values, effectively arranging 128 small feature maps on a tiny 7-pixel grid. The tensor is reshaped accordingly, after which *batch normalisation* centres and scales activations for steadier training, and a *Leaky ReLU* introduces gentle non-linearity so the model can learn richer patterns.

The next two layers are *transposed convolutions*, sometimes called "deconvolutions", whose task is to enlarge the image. 

The first transposed convolution uses 64 filters, a kernel size of 5 and a stride of 2, doubling each spatial dimension from 7 X 7 to 14 X 14 while learning to fill in texture; batch normalisation and Leaky ReLU follow once more. 

A second transposed convolution, also with a 5 X 5 kernel and stride 2, performs the final upscaling to 28 X 28 pixels and reduces the channel count to one, matching the grey-scale format of Fashion-MNIST. Its activation is `tanh`, which outputs values between −1 and +1 so that the generator’s range aligns with the rescaled training images. 

Explicit biases are omitted in layers that feed directly into batch normalisation, because the normalisation step already provides its own trainable offset. At the end of the function the fully assembled *Sequential* model is returned and assigned to `generator`, ready to take random noise and turn it into synthetic garments:

In [ ]:
from tensorflow.keras import layers

def build_generator():
    # Sequential container, layers run one after another
    return keras.Sequential([
        # Latent vector input, length = LATENT_DIM
        layers.Input(shape=(LATENT_DIM,)),

        # Fully-connected layer expands the 1-D seed to 7X7X128 units
        layers.Dense(7 * 7 * 128, use_bias=False),

        # Rearrange the flat vector into a stack of 128 feature maps, each 7X7
        layers.Reshape((7, 7, 128)),

        # Normalise activations for more stable gradients
        layers.BatchNormalization(),
        # Leaky ReLU adds a small negative slope, avoids "dead" neurons
        layers.LeakyReLU(0.2),

        # First transposed-convolution, doubles width and height to 14X14
        layers.Conv2DTranspose(
            64,              # number of output channels
            kernel_size=5,    # 5X5 filter
            strides=2,        # upscale by a factor of 2
            padding='same',
            use_bias=False    # bias not needed, BatchNorm follows
        ),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        # Second transposed-convolution, upsamples to 28X28 and outputs 1 channel
        layers.Conv2DTranspose(
            1,               # grey-scale output
            kernel_size=5,
            strides=2,
            padding='same',
            activation='tanh',  # match image range (-1 to +1)
            use_bias=False
        )
    ])

# Build the generator once so it is ready to receive latent vectors
generator = build_generator()


### Discriminator
The `build_discriminator` function constructs the critic that decides whether an input image looks authentic. It produces a *Sequential* model whose layers execute in order, which we then return. In terms of the structure, we have the following layers:

#### Input layer
The network expects a grey-scale picture shaped `(28, 28, 1)`, which matches the Fashion-MNIST format.

#### First convolution block
`Conv2D(64, 5, strides=2, padding='same')` applies 64 filters of size 5 X 5.
The stride of 2 halves the spatial resolution to 14 X 14 while detecting basic features such as edges and corners. A *Leaky ReLU* activation introduces non-linearity, and `Dropout(0.3)` randomly switches off 30 percent of activations during training, reducing over-fitting and preventing the discriminator from becoming too confident too quickly.

#### Second convolution block
Another convolution, now with 128 filters, repeats the process, shrinking the map to 7 X 7 and capturing more complex motifs like sleeves or laces. A second Leaky ReLU and Dropout follow for the same reasons as above.

#### Flatten and decision layer
`Flatten()` converts the 7 X 7 X 128 activation cube into a single vector. The final `Dense(1)` layer outputs one scalar *logit*: a positive value suggests the image is real, a negative value suggests it is fake. There is no sigmoid here because the training loss (`BinaryCrossentropy(from_logits=True)`) expects raw logits and is numerically more stable that way.

The statement `discriminator = build_discriminator()` instantiates this model so it can immediately begin judging samples produced by the generator or drawn from the true Fashion-MNIST data.


In [ ]:
def build_discriminator():
    
    # Sequential container, layers executed in order
    return keras.Sequential([
        # Input expects a 28x28 grey-scale image
        layers.Input(shape=(28, 28, 1)),

        # Convolution block 1: detect low-level features, downsample to 14X14
        layers.Conv2D(
            64,           # number of filters
            kernel_size=5,
            strides=2,    # divide width and height by 2
            padding='same'
        ),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),  # regularise so Discrimantor does not overfit too fast

        # Convolution block 2: deeper features, downsample to 7X7
        layers.Conv2D(
            128,
            kernel_size=5,
            strides=2,
            padding='same'
        ),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),

        # Flatten to a single vector
        layers.Flatten(),

        # Final dense layer outputs one logit; >0 suggests "real", <0 "fake"
        layers.Dense(1)
    ])

# Build the discriminator so it is ready to score images
discriminator = build_discriminator()


### Optimisers
`BinaryCrossentropy(from_logits=True)` creates the loss function that drives the adversarial game. Binary cross-entropy measures how well the discriminator’s single output value classifies an image as real or fake.

Setting `from_logits=True` tells Keras that the discriminator’s final layer returns raw scores (logits) rather than values already passed through a sigmoid. The loss routine therefore applies its own internal sigmoid, which is numerically more stable.

Both the generator and the discriminator are trained with the Adam optimiser, constructed here with identical hyper-parameters.

The learning rate `2e-4` is the figure found in the original DCGAN paper to balance speed and stability. You can read the original paper to learn why this might be the case.

The momentum term `beta_1` is lowered to 0.5 instead of the default 0.9, which helps prevent the optimiser from running too far in the wrong direction when the loss surface is volatile. `beta_2` remains at 0.999, maintaining the long-term average of squared gradients. Using the same optimiser settings for both networks keeps their learning dynamics in step during training:

In [ ]:
# Loss function: binary cross-entropy that expects raw logits from the discriminator
cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)

# Adam optimiser for the generator, LR = 0.0002, betas recommended in the DCGAN paper
gen_opt = keras.optimizers.Adam(2e-4, beta_1=0.5, beta_2=0.999)

# Same optimiser settings for the discriminator to keep both networks learning at similar rates
disc_opt = keras.optimizers.Adam(2e-4, beta_1=0.5, beta_2=0.999)

### Model training
The decorator `@tf.function` instructs TensorFlow to convert the Python routine that follows into a compact computation graph the first time it is called. Running subsequent steps through this graph removes much of the overhead normally associated with executing Python line by line, which leads to a significant speed-up, especially on a GPU.

Within `train_step` a single training round unfolds. The function first draws a batch of random latent vectors, each one a one-hundred-element column of Gaussian noise. These vectors serve as the generator’s raw creative material. Next comes the discriminator update. TensorFlow’s `GradientTape` watches the forward pass so that it can later calculate derivatives. The generator turns the noise into synthetic images; the discriminator then evaluates both these fakes and the genuine images supplied as `real_imgs`. Real outputs are compared with a target value of one, while fake outputs are compared with zero. 

The binary-cross-entropy loss from both parts is summed to form `d_loss`. Gradients of this loss with respect to every discriminator weight are computed and the optimiser applies them, nudging the discriminator to sharpen its ability to tell real from fake.

After the critic has been trained, attention shifts to the artist. The loop deliberately gives the generator two updates for every single discriminator update, a common tactic that helps the weaker network keep pace. For each generator pass fresh noise is sampled and converted into new images. These fakes are fed through the freshly updated discriminator, but this time the target label is one, because the generator’s objective is to persuade the discriminator that its creations are authentic. 

The resulting `g_loss` measures that success, gradients are taken with respect to the generator’s parameters, and the optimiser adjusts them accordingly. If we return both `d_loss` and `g_loss`, then we can log progress and diagnose stalled learning during the outer training loop:

In [ ]:
# Convert this Python function into a TensorFlow static graph for faster execution
@tf.function
def train_step(real_imgs):
    """
    One training step for GAN:
    - Updates the discriminator once.
    - Updates the generator twice (to help it catch up with discriminator).
    """

    # Sample a batch of latent vectors from a standard normal distribution
    noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])

    # Discriminator Update
    with tf.GradientTape() as tape:
        # Generate fake images from noise using the generator
        fake_imgs = generator(noise, training=True)

        # Get discriminator outputs (predicted real/fake scores) for real and fake images
        real_out = discriminator(real_imgs, training=True)
        fake_out = discriminator(fake_imgs, training=True)

        # Discriminator loss: real images should be classified as 1, fake as 0
        d_loss = cross_entropy(tf.ones_like(real_out), real_out) + \
                 cross_entropy(tf.zeros_like(fake_out), fake_out)

    # Compute gradients for discriminator parameters and apply update
    grads = tape.gradient(d_loss, discriminator.trainable_variables)
    disc_opt.apply_gradients(zip(grads, discriminator.trainable_variables))

    # Generator Update (run twice to maintain balance)
    for _ in range(2):
        # Sample a new batch of noise for each generator update
        noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])

        with tf.GradientTape() as tape:
            # Generate fake images
            fake_imgs = generator(noise, training=True)
            # Get discriminator scores for the fake images
            fake_out = discriminator(fake_imgs, training=True)

            # Generator loss: try to fool the discriminator (labels = 1 for fake images)
            g_loss = cross_entropy(tf.ones_like(fake_out), fake_out)

        # Compute gradients for generator parameters and apply update
        grads = tape.gradient(g_loss, generator.trainable_variables)
        gen_opt.apply_gradients(zip(grads, generator.trainable_variables))

    # Return both losses for monitoring
    return d_loss, g_loss


Next we create a function `show_generated` as a helper routine that lets us see what the generator has learned at any point during training. It begins by sampling `n` random latent vectors, each of length `LATENT_DIM`. These vectors are the generator’s raw material and, once fed into the model, produce `n` synthetic images. Because both the training data and the generator’s `tanh` output were scaled to the range −1 to +1, the code rescales the images back to 0 – 1 by adding 1 and dividing by 2, which is the range expected by Matplotlib’s grey‐scale display. This provides us with an at-a-glance gallery of the generator’s latest creations to visually evaluate the training performance:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def show_generated(step, n=16):
    # Draw n random latent vectors
    noise = tf.random.normal([n, LATENT_DIM])

    # Produce fake images and convert them back to the [0, 1] range
    imgs = (generator(noise, training=False) + 1) / 2

    # Setup plots in a square grid
    r = int(np.sqrt(n))
    plt.figure(figsize=(r, r))

    for i in range(n):
        plt.subplot(r, r, i + 1)  # Select position i in the grid
        plt.imshow(imgs[i, :, :, 0], cmap="gray")
        plt.axis("off")                       

    # Title showing which training step produced these images
    plt.suptitle(f"Step {step}")
    plt.show()

We create a function `train`, an outer loop that drives the whole adversarial process. The argument `steps` sets how many updates to run, by default thirty thousand, while `sample_int` decides how often to display progress.

The line `for step, real_batch in zip(range(steps), train_ds.repeat()):` pairs an integer counter with an endless stream of training images. 

Next, `train_ds.repeat()` creates an infinite dataset by cycling through the images over and over. `range(steps)` is finite, so `zip` truncates the combination after the chosen number of steps. In each pass the loop retrieves one mini-batch of real Fashion-MNIST pictures and sends it to `train_step`, which performs a single discriminator update and two generator updates. The function returns the current discriminator loss `d_loss` and generator loss `g_loss`.

Every `sample_int` steps the code prints a concise progress report, formatting the losses to four decimal places, then calls `show_generated(step)` to display a grid of fresh images from the generator:

In [ ]:
def train(steps=30000, sample_int=1000):
    # Loop over a fixed number of steps, pairing each step
    # with a batch of real images from an endlessly repeated dataset
    for step, real_batch in zip(range(steps), train_ds.repeat()):

        # Perform one training round: update D once, G twice
        d_loss, g_loss = train_step(real_batch)

        # Every sample_int steps, print losses and show generated samples
        if step % sample_int == 0:
            print(f'{step}: D={d_loss:.4f} | G={g_loss:.4f}')
            show_generated(step)
    
    print('Training complete.')
    print(f"Total steps: {steps:,}")
    print(f"Final losses - D: {d_loss:.4f},  G: {g_loss:.4f}")

    # Display one last batch of samples
    show_generated("final")

In [ ]:
# Start training
train(steps=30000, sample_int=500)

When the training loop ends we print a brief report, stating the total number of steps, and the final discriminator and generator loss values, which offers a snapshot of how well the two networks have balanced by the end. We also produce one last grid of synthetic images to inspect the quality of the generator's best output after all training iterations.

### What have we learnt?
We have seen how a DCGAN handles images effectively, and why normalising data to the range −1 to +1 matters when the generator finishes with a tanh activation. We unpacked Fashion-MNIST, which offers seventy thousand 28 X 28 grey-scale garments in ten balanced classes. This is a perfect dataset for rapid prototyping on limited hardware. We saw how adding an extra "channel" dimension prepares grey-scale data for convolution layers.

Inside the model we traced the generator's route from a 100-number latent vector to a 28 X 28 image, noting the role of transposed convolutions for up-scaling and batch normalisation for numerical stability. We then parsed the discriminator, whose job is the inverse: down-scaling images with ordinary convolutions and issuing a single logit that signals real or fake. We also examined the adversarial loss and the Adam optimiser settings that keep both networks learning at comparable speeds.

At the training loop level we saw how `@tf.function` compiles Python into a fast TensorFlow graph, how one discriminator step followed by two generator steps helps maintain balance, and why periodic sampling of newly generated images is crucial for diagnosing mode collapse early. 

We used black and white images as our training data, but many larger generative models use colour imageswith three channels, this is likely beyond the hardware you have available. However, hopefully this has provided a good demonstration on the power of generative models.